## GAT

In [1]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd
import cudf

data = pd.read_csv(r"extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


In [2]:
data = pd.concat((data_analysis.get_training_data(), data_analysis.get_testing_data()))
data = cudf.from_pandas(data)

In [3]:
data = data.sort_values("Id")

In [4]:
unique_ids = data["Id"].unique()

In [5]:
data = data.reset_index(drop=True)

In [6]:
data.pop("index")

0           665587
1           665588
2           665589
3           665590
4           665591
            ...   
1381353    2709538
1381354    2709541
1381355    2709540
1381356    2709539
1381357    2709536
Name: index, Length: 1381358, dtype: int64

In [7]:
data

,Id,Age,Sex,Diagnosis,Center,Set,Sender,Episode,Time,TargetIcu,SecToIcu,CRP,HGB,MCV,PCT,PLT,RBC,WBC,Label
0,1,79,M,Control,Leipzig,Training,ED,1,420.0,<NA>,<NA>,0.75,8.1,86.0,<NA>,167.0,4.36,7.3,Control
1,2,35,M,Control,Leipzig,Training,GEN,1,0.0,<NA>,<NA>,<NA>,10.6,79.9,<NA>,199.0,6.02,4.7,Control
2,3,47,W,Control,Leipzig,Training,AMB,1,0.0,<NA>,<NA>,3.87,8.7,89.9,<NA>,298.0,4.37,7.1,Control
3,4,28,M,Control,Leipzig,Training,ED,1,780.0,<NA>,<NA>,0.52,9.9,96.2,<NA>,216.0,4.79,5.3,Control
4,5,63,W,Control,Leipzig,Training,GEN,1,0.0,<NA>,<NA>,232.87,7.4,86.6,<NA>,189.0,3.96,20.9,Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381353,778980,56,M,Sepsis,Leipzig,Validation,GEN,1,606720.0,MICU,296040.0,17.88,6.7,96.6,<NA>,121.0,3.20,7.2,Control
1381354,778980,56,M,Sepsis,Leipzig,Validation,GEN,1,895680.0,MICU,7080.0,22.98,7.7,91.8,<NA>,155.0,3.67,7.1,Sepsis
1381355,778980,56,M,Sepsis,Leipzig,Validation,GEN,1,888540.0,MICU,14220.0,19.26,6.6,91.1,<NA>,149.0,3.14,6.5,Sepsis
1381356,778980,56,M,Sepsis,Leipzig,Validation,GEN,1,870420.0,MICU,32340.0,20.46,6.5,93.2,<NA>,114.0,3.07,8.3,Control


In [8]:
from dataAnalysis.Constants import SEX_CATEGORY_COLUMN_NAME, SEX_COLUMN_NAME, FEATURES
data[SEX_CATEGORY_COLUMN_NAME] = data.loc[:, SEX_COLUMN_NAME] =="W"

In [9]:
data[SEX_CATEGORY_COLUMN_NAME] = data[SEX_CATEGORY_COLUMN_NAME].astype("int8")
data["Label"] = data["Label"] == "Sepsis"
data["Label"] = data["Label"].astype("int8")

In [10]:
import numpy as np
import matplotlib.pyplot as plt

def getPositionEncoding(seq_len, d = len(FEATURES), n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P

In [11]:
from dataAnalysis.Constants import FEATURES
for feature in FEATURES:
    data[feature] = (data[feature] - data[feature].mean()) / data[feature].std()

In [12]:
np.random.seed(42)
random = np.random.uniform(0, 1 ,data.shape[0])
data["random"] = random

In [ ]:
import numpy as np
import cupy as cp
import time
from torch_geometric.data import Data
import torch

graph_list = []
start = time.time()
i = 0
for Id, group in data.groupby("Id"):
    i+=1
    if i % 1000 == 0:
        print(f"{str(i / unique_ids.shape[0] * 100)} %")

print(time.time() - start)

In [ ]:
import threading
class Fred(threading.Thread):
    def __init__(self, Id, it):
        threading.Thread.__init__(self)
        self.id = Id
        self.it = it
    
    def run(self):
        self.id
        self.it = self.it+1
        

import numpy as np
import cupy as cp
import time
from torch_geometric.data import Data
import torch

graph_list = []
start = time.time()
i = 0
for Id, group in data.groupby("Id"):
    fred = Fred(Id, i)
    fred.start()
    i = fred.it
    if i % 1000 == 0:
        print(f"{str(i / unique_ids.shape[0] * 100)} %")

print(time.time() - start)

In [13]:
import numpy as np
import cupy as cp
import time
from torch_geometric.data import Data
import torch
import threading

class Edgy_Fred(threading.Thread):
    def __init__(self, indices):
        threading.Thread.__init__(self)
        self.indices = indices
        self.edge_index = None
    
    def run(self):
        indices = cp.expand_dims(self.indices.values, axis = 1)
        target = cp.repeat(indices, indices.shape[0], axis = 1)
        source = target.transpose().flatten()
        target = target.flatten()
        self.edge_index = np.asarray([np.asarray(source.get()), np.asarray(target.get())])
        
        
class Encoding_Fred(threading.Thread):
    def __init__(self, group):
        threading.Thread.__init__(self)
        self.group = group
        self.encoding = None
    
    def run(self):
        self.encoding = getPositionEncoding(self.group.shape[0])
        
class Masked_Fred(threading.Thread):
    def __init__(self, group):
        threading.Thread.__init__(self)
        self.group = group
        self.train_mask = None
        self.val_mask = None
        self.test_mask = None
    
    def run(self):
        train_mask_ser = (self.group["Set"] != "Validation").to_arrow().to_pylist()
        val_ratio_mask = (self.group["random"] >= 0.8)
        train_ratio_mask = (self.group["random"] < 0.8)
        self.val_mask = np.logical_and(train_mask_ser, val_ratio_mask.to_arrow().to_pylist())
        self.train_mask = np.logical_and(train_mask_ser, train_ratio_mask.to_arrow().to_pylist())    
        self.test_mask = (group["Set"] == "Validation").values.get()
        
class MainFred(threading.Thread):
    def __init__(self, group, graph_list):
        threading.Thread.__init__(self)
        self.group = group
        self.graph_list = graph_list
    
    def run(self):
        group = self.group.reset_index().sort_values("Time").reset_index()
        indices = group.index

        ## Edge construction
        edgy_fred = Edgy_Fred(indices)
        edgy_fred.start()

        ## Features
        X_features = torch.tensor(group[FEATURES].values.get()).type(torch.float32)


        ## Positions
        encoding_fred = Encoding_Fred(group)
        encoding_fred.start()

        ## Masks
        masked_fred = Masked_Fred(group)
        masked_fred.start()
        
        edgy_fred.join()
        encoding_fred.join()
        masked_fred.join()
        ## Graph
        graph = Data(x= X_features + encoding_fred.encoding,
                     y= torch.tensor(group["Label"].values.get()).type(torch.int8),
                     edge_index= torch.tensor(edgy_fred.edge_index),
                     train_mask=torch.from_numpy(masked_fred.train_mask).type(torch.bool),
                     test_mask=torch.from_numpy(masked_fred.test_mask).type(torch.bool),
                     val_mask=torch.from_numpy(masked_fred.val_mask).type(torch.bool))
        self.graph_list.append(graph)

graph_list = []
start = time.time()
i = 0
for Id, group in data.groupby("Id"):
    group = group.reset_index().sort_values("Time").reset_index()
    indices = group.index

    ## Edge construction
    edgy_fred = Edgy_Fred(indices)
    edgy_fred.start()

    ## Features
    X_features = torch.tensor(group[FEATURES].values.get()).type(torch.float32)


    ## Positions
    encoding_fred = Encoding_Fred(group)
    encoding_fred.start()

    ## Masks
    masked_fred = Masked_Fred(group)
    masked_fred.start()

    edgy_fred.join()
    encoding_fred.join()
    masked_fred.join()
    ## Graph
    graph = Data(x= X_features + encoding_fred.encoding,
                 y= torch.tensor(group["Label"].values.get()).type(torch.int8),
                 edge_index= torch.tensor(edgy_fred.edge_index),
                 train_mask=torch.from_numpy(masked_fred.train_mask).type(torch.bool),
                 test_mask=torch.from_numpy(masked_fred.test_mask).type(torch.bool),
                 val_mask=torch.from_numpy(masked_fred.val_mask).type(torch.bool))
    graph_list.append(graph)
    i+=1
    if i % 1000 == 0:
        print(f"{str(i / unique_ids.shape[0] * 100)} %")

print(time.time() - start )

0.1411243375976404 %
0.2822486751952808 %
0.4233730127929212 %
0.5644973503905616 %
0.705621687988202 %
0.8467460255858424 %
0.9878703631834829 %
1.1289947007811232 %
1.2701190383787637 %
1.411243375976404 %
1.5523677135740444 %
1.6934920511716849 %
1.8346163887693252 %
1.9757407263669657 %
2.116865063964606 %
2.2579894015622464 %
2.399113739159887 %
2.5402380767575274 %
2.6813624143551675 %
2.822486751952808 %
2.963611089550448 %
3.1047354271480887 %
3.2458597647457297 %
3.3869841023433698 %
3.52810843994101 %
3.6692327775386504 %
3.810357115136291 %
3.9514814527339315 %
4.092605790331572 %
4.233730127929212 %
4.374854465526853 %
4.515978803124493 %
4.657103140722133 %
4.798227478319774 %
4.939351815917414 %
5.080476153515055 %
5.221600491112695 %
5.362724828710335 %
5.503849166307976 %
5.644973503905616 %
5.786097841503256 %
5.927222179100896 %
6.068346516698537 %
6.209470854296177 %
6.350595191893818 %
6.491719529491459 %
6.632843867089099 %
6.7739682046867395 %
6.915092542284379 %


56.87310805184909 %
57.01423238944672 %
57.15535672704436 %
57.296481064642 %
57.43760540223965 %
57.578729739837286 %
57.71985407743493 %
57.86097841503256 %
58.0021027526302 %
58.143227090227846 %
58.284351427825484 %
58.42547576542313 %
58.566600103020775 %
58.70772444061841 %
58.848848778216045 %
58.98997311581369 %
59.13109745341133 %
59.272221791008974 %
59.413346128606605 %
59.55447046620424 %
59.69559480380189 %
59.83671914139953 %
59.97784347899717 %
60.11896781659482 %
60.26009215419245 %
60.40121649179009 %
60.54234082938773 %
60.68346516698537 %
60.82458950458302 %
60.965713842180655 %
61.106838179778286 %
61.24796251737593 %
61.38908685497357 %
61.530211192571215 %
61.67133553016886 %
61.8124598677665 %
61.95358420536413 %
62.094708542961776 %
62.235832880559414 %
62.37695721815706 %
62.5180815557547 %
62.65920589335234 %
62.800330230949974 %
62.94145456854762 %
63.08257890614526 %
63.223703243742904 %
63.36482758134054 %
63.50595191893819 %
63.64707625653582 %
63.78820059

In [14]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

WEIGHT = torch.tensor([664])
WEIGHT = WEIGHT.to(device)

print("Data shifted to the device " + str(device))

Data shifted to the device cuda:0


In [15]:
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv,GATv2Conv, GINConv, global_add_pool
from torch.nn import Linear
import torch
from dataAnalysis.Constants import FEATURES
from torch.nn import Linear, ReLU, Sequential
from torch.nn import BatchNorm1d as BatchNorm

class GraphNeuralNetwork(torch.nn.Module):
    def __init__(self, hidden_dim = 128, out_channels = 1):
        super(GraphNeuralNetwork, self).__init__()
        input_dim = len(FEATURES)      
        
        conv_1= GATConv(input_dim, hidden_dim,heads=1, add_self_loops = False)
        conv_end = GATConv((-1,-1), out_channels,add_self_loops = False)
        
        self.conv_1 = conv_1
        self.conv_end = conv_end
        

    def forward(self, graph):
        x, edge_index = graph.x, graph.edge_index
        x = x.type(torch.float)
        x = self.conv_1(x, edge_index)
        x = F.normalize(x, p=2., dim=-1)
        x = torch.relu(x)
        x = self.conv_end(x, edge_index)
        return x
            
    def predict_proba(self, graph, mask):
        with torch.inference_mode():
            self.eval()
            logits = self.forward(graph)
            scores = torch.sigmoid(torch.squeeze(logits[mask]))
            scores = torch.unsqueeze(scores, 0)
            proba_predict = torch.concat((1- scores, scores), dim = 0)
            return torch.transpose(proba_predict, 0, 1)
            
    def predict(self, graph, mask):
        return torch.round(self.predict_proba(graph, mask)[:, 1])

In [23]:
for graph in graph_list:
    graph.cpu()#.to(device)

In [26]:
from torch_geometric.loader import DataLoader

data_loader = DataLoader(graph_list, batch_size=100_000)

In [37]:
i = 0
for batch, graph in enumerate(data_loader):
    print(graph)
print(i)

DataBatch(x=[192030, 7], edge_index=[2, 1239426], y=[192030], train_mask=[192030], test_mask=[192030], val_mask=[192030], batch=[192030], ptr=[100001])
DataBatch(x=[192705, 7], edge_index=[2, 1433261], y=[192705], train_mask=[192705], test_mask=[192705], val_mask=[192705], batch=[192705], ptr=[100001])


KeyboardInterrupt: 

In [63]:
import torch 
# import wandb

LEARNING_RATE = 3e-4
MAX_EPOCHS = 100 #40000
HIDDEN_DIM = 128 #128
# wandb.init(
#     project="gat-time-analysis",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LEARNING_RATE,
#     "architecture": "2 GATConv,128 neurons,1 head,layer normalization,ReLU,early stopping (10)",
#     "dataset": "SBC",
#     "epochs": MAX_EPOCHS,
#     }
# )


class ModelWrapper():
    def __init__(self, data_loader):
        self.LEARNING_RATE = LEARNING_RATE
        self.MAX_EPOCHS = MAX_EPOCHS

        self.model = GraphNeuralNetwork(hidden_dim = 7, out_channels=1) 
        self.model = self.model.to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.LEARNING_RATE,betas=(0.9, 0.999), eps=1e-08)
        self.data_loader = data_loader
        
        self.last_loss = 0
        self.increased_loss = 0
        self.BREAKING_THRESHOLD = 10    
        self.val_loss = []
        self.train_loss = []
    
    def validate(self):
        with torch.inference_mode():
            for data in self.data_loader:
                data = data.to(device)
                self.model.eval()
                out = self.model(data)
                loss = F.binary_cross_entropy_with_logits(torch.squeeze(out[data.val_mask]), torch.squeeze(data.y[data.val_mask].type(torch.float32)),
                                                      pos_weight=torch.squeeze(WEIGHT))
                self.val_loss.append(loss.item())
                if loss.item() > self.last_loss:
                    self.increased_loss += 1
                else:
                    self.increased_loss = 0
                self.last_loss = loss.item()

    def train(self):
        for epoch in range(self.MAX_EPOCHS):
            for data in self.data_loader:
                data = data.to(device)
                self.model.train()
                self.optimizer.zero_grad()
                out = self.model(data)
                loss = F.binary_cross_entropy_with_logits(torch.squeeze(out[data.train_mask]), torch.squeeze(data.y[data.train_mask].type(torch.float32)),
                                                          pos_weight=torch.squeeze(WEIGHT))
                self.train_loss.append(loss.item())
#                 wandb.log({"loss": loss.item()})
                loss.backward()
                self.optimizer.step()
                self.validate() 
                if self.increased_loss >= self.BREAKING_THRESHOLD:
    #                 print(f"Breaked at {str(epoch)}")
                    break
            
    def get_model(self):
        return self.model    
    
    def plot_loss(self):
        plt.plot(self.epochs, self.train_loss, 'g', label='Training loss')
        plt.plot(self.epochs, self.val_loss, 'y', label='Validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

In [ ]:
model_wrapper = ModelWrapper(data_loader)
model_wrapper.train()

In [ ]:
model = model_wrapper.get_model()
graph = graph.cpu()
model = model.cpu()

In [ ]:
pred_logits = torch.tensor([])
true_val = torch.tensor([])
for data in data_loader:
    if data.test_mask.sum() == 0:
        continue
    with torch.inference_mode():
        model.eval()
        logits = model(data)
        test_logits = torch.sigmoid(logits[data.test_mask])
        y_true = data.y[data.test_mask]
        true_val = torch.concat((true_val, y_true))
        pred_logits = torch.concat((pred_logits, test_logits))

In [ ]:
pred_logits_sq = torch.squeeze(pred_logits)

In [ ]:
scores = torch.unsqueeze(pred_logits_sq, 0)
proba_predict = torch.concat((1- scores, scores), dim = 0)
y_proba = torch.transpose(proba_predict, 0, 1)
y_pred = torch.round(y_proba[:, 1])

In [ ]:
from dataAnalysis.Metrics import Evaluation
y_dict = Evaluation.create_y_dict(y_pred, y_proba , true_val)

In [ ]:
from dataAnalysis.Metrics import Evaluation

Evaluation.plot_confusion_matrix_from_pred(y_pred,true_val)
Evaluation.get_df_metrics_from_pred(y_dict, y_dict)

In [ ]:
# wandb.log({"acc": acc, "loss": loss})
# wandb.finish()

In [ ]:
from IPython.display import Audio
sound_file = './finish_sound.mp3'
Audio(sound_file, autoplay=True)

In [36]:
import time

for i in range(10000):
    time.sleep(60)
    print("Sleeping")

Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping


KeyboardInterrupt: 